### ITD
En este dataset tenemos fotos tomadas con 10 cámaras diferentes a diferentes telas. De cada cámara tenemos una parte de las fotos para train, en las que solamente hay fotos de telas en buen estado. Y otra parte para test, en las que hay fotos de telas en buen estado y dañadas.
El objetivo es entrenar el modelo solo con las fotos de train (tela en buen estado) y poder predecir si una foto de test es una tela en buen estado ($y_{test}=0$) o dañada ($y_{test}=1$).

## Leer datos todos juntos

Leemos mezclando test y train, y luego particioanremos todos los datos en train, val y test

In [15]:
import cv2
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

def cargar_dataset_imagenes(root_path, image_size=(64, 64)):
    """
    Lee las imágenes y las devuelve en formato (N, Alto, Ancho).
    NO aplana las imágenes. Mantiene la estructura espacial.
    """
    X_list = []
    y_list = []
    
    root = Path(root_path)
    print(f"📂 Escaneando imágenes en: {root.absolute()} ...")
    
    if not root.exists():
        print("❌ Error: La ruta no existe.")
        return np.array([]), np.array([])

    contador = 0
    
    for img_path in root.rglob('*.*'):
        if img_path.suffix.lower() not in ['.png', '.jpg', '.jpeg', '.bmp']:
            continue
            
        # --- 1. ETIQUETADO ---
        # 1 = Defecto (Anomaly), 0 = Bien (Good)
        label = 1 if 'anomaly' in str(img_path).lower() else 0
            
        # --- 2. LECTURA (Escala de Grises) ---
        # Leemos en blanco y negro para simplificar (1 canal). 
        # Si quisieras color, quita el flag IMREAD_GRAYSCALE.
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        
        if img is None: continue
        
        # --- 3. RESIZE (Obligatorio) ---
        # Todas deben medir lo mismo para entrar en el array numpy
        img = cv2.resize(img, image_size)
        
        # --- 4. NORMALIZACIÓN ---
        # Pasamos de 0-255 (enteros) a 0.0-1.0 (float).
        # Esto es vital tanto para ML clásico como para Deep Learning.
        img_norm = img.astype('float32') / 255.0
        
        X_list.append(img_norm)
        y_list.append(label)
        contador += 1

    # Convertimos la lista a un Array Numpy 3D: (N_fotos, Alto, Ancho)
    X_array = np.array(X_list)
    y_array = np.array(y_list)

    print(f"✅ Carga completa. {contador} imágenes procesadas.")
    return X_array, y_array

# --- EJECUCIÓN: CARGAR Y DIVIDIR ---

ruta = "./ITD" # Tu ruta
X_total, y_total = cargar_dataset_imagenes(ruta, image_size=(64, 64))

# Verificación de forma
# Debería salir algo como: (5000, 64, 64) -> 5000 fotos de 64x64
print(f"\nForma de X_total: {X_total.shape}") 

# --- PARTICIÓN (Train / Val / Test) ---
# 1. Separamos Train (70%) del resto (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_total, y_total, test_size=0.3, random_state=42, stratify=y_total
)

# 2. Separamos el resto en Val (15%) y Test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("-" * 30)
print(f"🔹 Train shape: {X_train.shape} (Listo para Autoencoders/CNN)")
print(f"🔹 Val shape:   {X_val.shape}")
print(f"🔹 Test shape:  {X_test.shape}")
print("-" * 30)

# mostramos numero de anomalias en cada conjunto
print(f"Train anomalies: {np.sum(y_train)}")
print(f"Val anomalies:   {np.sum(y_val)}")
print(f"Test anomalies:  {np.sum(y_test)}")

📂 Escaneando imágenes en: c:\Users\ander\OneDrive - UPNA\4º\Extracción del conocimiento\TrabajoGrupo\ITD ...
✅ Carga completa. 5878 imágenes procesadas.

Forma de X_total: (5878, 64, 64)
------------------------------
🔹 Train shape: (4114, 64, 64) (Listo para Autoencoders/CNN)
🔹 Val shape:   (882, 64, 64)
🔹 Test shape:  (882, 64, 64)
------------------------------
Train anomalies: 675
Val anomalies:   145
Test anomalies:  145


Ahora la forma de leerlos de train (solo good) y test (good y anomaly) por separado:

In [ ]:
import cv2
import numpy as np
from pathlib import Path

def cargar_datos_separados_raw(root_path, image_size=(64, 64)):
    """
    Lee las carpetas originales 'train' y 'test' y devuelve los datos separados.
    Formato: RAW (Píxeles 2D, normalizados 0-1).
    """
    # Listas para Entrenamiento
    X_train = []
    y_train = []
    
    # Listas para Test
    X_test = []
    y_test = []
    
    root = Path(root_path)
    print(f"📂 Escaneando estructura original en: {root.absolute()} ...")
    
    if not root.exists():
        print("❌ Error: La ruta no existe.")
        return np.array([]), np.array([]), np.array([]), np.array([])

    contador_train = 0
    contador_test = 0
    
    for img_path in root.rglob('*.*'):
        if img_path.suffix.lower() not in ['.png', '.jpg', '.jpeg', '.bmp']:
            continue
        
        path_str = str(img_path).lower()
        
        # --- 1. IDENTIFICAR SPLIT (¿Es Train o Test?) ---
        if 'train' in path_str:
            is_train = True
        elif 'test' in path_str:
            is_train = False
        else:
            continue # Si no está en ninguna carpeta train/test, la ignoramos
            
        # --- 2. IDENTIFICAR ETIQUETA (¿Good o Anomaly?) ---
        # 1 = Defecto, 0 = Bien
        label = 1 if 'anomaly' in path_str else 0
        
        # --- 3. LEER RAW (Escala de Grises) ---
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        
        if img is None: continue
        
        # --- 4. RESIZE & NORMALIZACIÓN ---
        img = cv2.resize(img, image_size)
        img_norm = img.astype('float32') / 255.0
        
        # --- 5. GUARDAR DONDE TOQUE ---
        if is_train:
            X_train.append(img_norm)
            y_train.append(label)
            contador_train += 1
        else:
            X_test.append(img_norm)
            y_test.append(label)
            contador_test += 1

    # Convertir todo a Numpy Arrays
    X_train_arr = np.array(X_train)
    y_train_arr = np.array(y_train)
    X_test_arr = np.array(X_test)
    y_test_arr = np.array(y_test)

    print(f"✅ Carga finalizada.")
    print(f"   Train: {contador_train} imágenes.")
    print(f"   Test:  {contador_test} imágenes.")
    
    return X_train_arr, y_train_arr, X_test_arr, y_test_arr

# --- USO ---
ruta = "./ITD" # Asegúrate que es la ruta correcta
X_train, y_train, X_test, y_test = cargar_datos_separados_raw(ruta, image_size=(64, 64))

print("\n--- DIMENSIONES (RAW) ---")
# Debería salir (N, 64, 64) -> Datos 2D listos para lo que quieras
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test:  {X_test.shape}")
print(f"y_test:  {y_test.shape}")

In [ ]:
# Ejemplo rápido para Isolation Forest
# -1 le dice a numpy: "calcula tú esta dimensión" (que será 64*64 = 4096)
X_train_flat = X_train.reshape(X_train.shape[0], -1) 
X_test_flat = X_test.reshape(X_test.shape[0], -1)

print(f"Forma aplanada para ML Clásico: {X_train_flat.shape}") 
# Resultado: (N, 4096)

# clf.fit(X_train_flat) ...

In [ ]:
# Ejemplo mental para futuro (PyTorch)
# Solo tendrías que añadir una dimensión de canal: (N, 1, 64, 64)
# tensor_img = torch.from_numpy(X_train).unsqueeze(1)